{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Análisis Exploratorio de Datos (EDA) - Ruleta de la Vida\n",
    "\n",
    "Este notebook documenta el análisis exploratorio de los datos recopilados a través de la aplicación \"Ruleta de la Vida\". El objetivo es entender las distribuciones, patrones y relaciones en los datos para informar el desarrollo de modelos predictivos y obtener insights accionables.\n",
    "\n",
    "## Contenido\n",
    "1. Carga y Preparación de Datos\n",
    "2. Estadísticas Descriptivas\n",
    "3. Análisis de Distribuciones\n",
    "4. Análisis por Categorías\n",
    "5. Análisis Demográfico\n",
    "6. Análisis Temporal\n",
    "7. Correlaciones y Relaciones\n",
    "8. Insights y Conclusiones"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Carga y Preparación de Datos\n",
    "\n",
    "Primero, importamos las bibliotecas necesarias y cargamos los datos desde la base de datos SQLite."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Importar bibliotecas necesarias\n",
    "import sqlite3\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from datetime import datetime, timedelta\n",
    "import warnings\n",
    "\n",
    "# Ignorar advertencias para una salida más limpia\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Configurar estilo de visualización\n",
    "sns.set(style=\"whitegrid\")\n",
    "plt.style.use('seaborn-v0_8-whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 8)\n",
    "plt.rcParams['font.size'] = 12\n",
    "plt.rcParams['axes.titlesize'] = 16\n",
    "plt.rcParams['axes.labelsize'] = 14"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Cargar datos desde la base de datos SQLite\n",
    "def cargar_datos():\n",
    "    try:\n",
    "        conn = sqlite3.connect('../database.db')\n",
    "        \n",
    "        # Consulta para obtener datos de respuestas junto con información de usuarios\n",
    "        query = \"\"\"\n",
    "        SELECT r.*, u.username \n",
    "        FROM respuestas r\n",
    "        JOIN usuarios u ON r.usuario_id = u.id\n",
    "        \"\"\"\n",
    "        \n",
    "        df = pd.read_sql_query(query, conn)\n",
    "        conn.close()\n",
    "        return df\n",
    "    except Exception as e:\n",
    "        print(f\"Error al cargar datos: {e}\")\n",
    "        # Si hay un error, crear un DataFrame de ejemplo para desarrollo\n",
    "        return crear_datos_ejemplo()\n",
    "\n",
    "# Función para crear datos de ejemplo en caso de que no se pueda acceder a la base de datos\n",
    "def crear_datos_ejemplo():\n",
    "    print(\"Creando datos de ejemplo para desarrollo...\")\n",
    "    \n",
    "    # Categorías y preguntas\n",
    "    categorias = ['salud_y_bienestar', 'relaciones', 'carrera_y_proposito', 'finanzas', \n",
    "                  'desarrollo_personal_y_crecimiento', 'diversion_y_ocio', 'espiritualidad', 'entorno_fisico_y_hogar']\n",
    "    \n",
    "    # Crear datos de ejemplo\n",
    "    data = []\n",
    "    for user_id in range(1, 101):  # 100 usuarios\n",
    "        username = f\"usuario_{user_id}\"\n",
    "        nombre = f\"Nombre {user_id}\"\n",
    "        edad = np.random.randint(18, 65)\n",
    "        sexo = np.random.choice(['masculino', 'femenino', 'otro'], p=[0.48, 0.48, 0.04])\n",
    "        estado_civil = np.random.choice(['soltero', 'casado', 'union libre', 'divorciado', 'viudo'], \n",
    "                                        p=[0.45, 0.35, 0.1, 0.08, 0.02])\n",
    "        \n",
    "        for categoria in categorias:\n",
    "            for pregunta_num in range(1, 4):  # 3 preguntas por categoría\n",
    "                pregunta = f\"Pregunta {pregunta_num} de {categoria}\"\n",
    "                \n",
    "                # Simular tendencias en las calificaciones basadas en características demográficas\n",
    "                base_calificacion = np.random.normal(5, 1.5)\n",
    "                \n",
    "                # Ajustar calificación según edad (personas mayores tienden a calificar más alto en espiritualidad)\n",
    "                if categoria == 'espiritualidad' and edad > 50:\n",
    "                    base_calificacion += 1.5\n",
    "                \n",
    "                # Ajustar calificación según estado civil (casados tienden a calificar más alto en relaciones)\n",
    "                if categoria == 'relaciones' and estado_civil in ['casado', 'union libre']:\n",
    "                    base_calificacion += 1\n",
    "                \n",
    "                # Ajustar calificación según sexo (tendencias hipotéticas para el ejemplo)\n",
    "                if categoria == 'desarrollo_personal_y_crecimiento' and sexo == 'femenino':\n",
    "                    base_calificacion += 0.5\n",
    "                \n",
    "                # Limitar calificación entre 1 y 10\n",
    "                calificacion = max(1, min(10, round(base_calificacion)))\n",
    "                \n",
    "                # Simular fecha (últimos 6 meses)\n",
    "                dias_atras = np.random.randint(0, 180)\n",
    "                fecha = pd.Timestamp.now() - pd.Timedelta(days=dias_atras)\n",
    "                \n",
    "                data.append({\n",
    "                    'id': len(data) + 1,\n",
    "                    'usuario_id': user_id,\n",
    "                    'nombre': nombre,\n",
    "                    'edad': edad,\n",
    "                    'sexo': sexo,\n",
    "                    'estado_civil': estado_civil,\n",
    "                    'categoria': categoria,\n",
    "                    'pregunta': pregunta,\n",
    "                    'calificacion': calificacion,\n",
    "                    'fecha': fecha,\n",
    "                    'username': username\n",
    "                })\n",
    "    \n",
    "    return pd.DataFrame(data)\n",
    "\n",
    "# Cargar los datos\n",
    "df = cargar_datos()\n",
    "print(f\"Datos cargados: {df.shape[0]} filas y {df.shape[1]} columnas\")\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Preparación de datos\n",
    "def preparar_datos(df):\n",
    "    # Convertir fecha a datetime si no lo es ya\n",
    "    if not pd.api.types.is_datetime64_any_dtype(df['fecha']):\n",
    "        df['fecha'] = pd.to_datetime(df['fecha'])\n",
    "    \n",
    "    # Extraer características de la fecha\n",
    "    df['fecha_solo'] = df['fecha'].dt.date\n",
    "    df['mes'] = df['fecha'].dt.month\n",
    "    df['dia_semana'] = df['fecha'].dt.dayofweek\n",
    "    df['es_fin_semana'] = df['dia_semana'].apply(lambda x: 1 if x >= 5 else 0)\n",
    "    \n",
    "    # Crear grupos de edad\n",
    "    bins = [0, 25, 35, 45, 55, 100]\n",
    "    labels = ['18-25', '26-35', '36-45', '46-55', '56+']\n",
    "    df['grupo_edad'] = pd.cut(df['edad'], bins=bins, labels=labels, right=False)\n",
    "    \n",
    "    # Verificar valores nulos\n",
    "    print(\"\\nValores nulos por columna:\")\n",
    "    print(df.isnull().sum())\n",
    "    \n",
    "    # Eliminar filas con valores nulos en calificación (si hay alguna)\n",
    "    df = df.dropna(subset=['calificacion'])\n",
    "    \n",
    "    return df\n",
    "\n",
    "# Preparar los datos\n",
    "df = preparar_datos(df)\n",
    "print(\"\\nColumnas después de la preparación:\")\n",
    "print(df.columns.tolist())\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Estadísticas Descriptivas\n",
    "\n",
    "Analizamos las estadísticas básicas de nuestros datos para entender su distribución y características."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Estadísticas descriptivas generales\n",
    "print(\"Estadísticas descriptivas de las calificaciones:\")\n",
    "print(df['calificacion'].describe())\n",
    "\n",
    "# Información demográfica\n",
    "print(\"\\nDistribución por sexo:\")\n",
    "print(df['sexo'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%')\n",
    "\n",
    "print(\"\\nDistribución por estado civil:\")\n",
    "print(df['estado_civil'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%')\n",
    "\n",
    "print(\"\\nDistribución por grupo de edad:\")\n",
    "print(df['grupo_edad'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%')\n",
    "\n",
    "# Estadísticas por categoría\n",
    "print(\"\\nEstadísticas por categoría:\")\n",
    "categoria_stats = df.groupby('categoria')['calificacion'].agg(['count', 'mean', 'std', 'min', 'max']).round(2)\n",
    "print(categoria_stats.sort_values('mean', ascending=False))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Estadísticas por pregunta\n",
    "pregunta_stats = df.groupby(['categoria', 'pregunta'])['calificacion'].agg(['count', 'mean', 'std']).round(2)\n",
    "print(\"Estadísticas por pregunta (top 10 con mayor y menor calificación):\")\n",
    "print(\"\\nTop 10 preguntas con mayor calificación:\")\n",
    "print(pregunta_stats.sort_values('mean', ascending=False).head(10))\n",
    "print(\"\\nTop 10 preguntas con menor calificación:\")\n",
    "print(pregunta_stats.sort_values('mean').head(10))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Análisis de Distribuciones\n",
    "\n",
    "Visualizamos las distribuciones de las calificaciones y otras variables clave."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Distribución general de calificaciones\n",
    "plt.figure(figsize=(12, 6))\n",
    "sns.histplot(df['calificacion'], kde=True, bins=10, color='steelblue')\n",
    "plt.axvline(df['calificacion'].mean(), color='red', linestyle='dashed', linewidth=2, label=f'Media: {df[\"calificacion\"].mean():.2f}')\n",
    "plt.axvline(df['calificacion'].median(), color='green', linestyle='dashed', linewidth=2, label=f'Mediana: {df[\"calificacion\"].median():.2f}')\n",
    "plt.title('Distribución de Calificaciones')\n",
    "plt.xlabel('Calificación')\n",
    "plt.ylabel('Frecuencia')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.show()\n",
    "\n",
    "# Distribución de calificaciones por categoría\n",
    "plt.figure(figsize=(14, 8))\n",
    "sns.boxplot(x='categoria', y='calificacion', data=df, palette='viridis')\n",
    "plt.title('Distribución de Calificaciones por Categoría')\n",
    "plt.xlabel('Categoría')\n",
    "plt.ylabel('Calificación')\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Distribución de edades\n",
    "plt.figure(figsize=(12, 6))\n",
    "sns.histplot(df['edad'].drop_duplicates(), kde=True, bins=15, color='darkorange')\n",
    "plt.title('Distribución de Edades')\n",
    "plt.xlabel('Edad')\n",
    "plt.ylabel('Frecuencia')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Distribución de calificaciones por categoría (violin plot)\n",
    "plt.figure(figsize=(14, 8))\n",
    "sns.violinplot(x='categoria', y='calificacion', data=df, palette='viridis', inner='quartile')\n",
    "plt.title('Distribución de Calificaciones por Categoría (Violin Plot)')\n",
    "plt.xlabel('Categoría')\n",
    "plt.ylabel('Calificación')\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Distribución de calificaciones por sexo\n",
    "plt.figure(figsize=(10, 6))\n",
    "sns.boxplot(x='sexo', y='calificacion', data=df, palette='Set2')\n",
    "plt.title('Distribución de Calificaciones por Sexo')\n",
    "plt.xlabel('Sexo')\n",
    "plt.ylabel('Calificación')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.show()\n",
    "\n",
    "# Distribución de calificaciones por estado civil\n",
    "plt.figure(figsize=(12, 6))\n",
    "sns.boxplot(x='estado_civil', y='calificacion', data=df, palette='Set3')\n",
    "plt.title('Distribución de Calificaciones por Estado Civil')\n",
    "plt.xlabel('Estado Civil')\n",
    "plt.ylabel('Calificación')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Análisis por Categorías\n",
    "\n",
    "Profundizamos en el análisis de las diferentes categorías de la Ruleta de la Vida."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calificaciones promedio por categoría\n",
    "plt.figure(figsize=(12, 6))\n",
    "cat_avg = df.groupby('categoria')['calificacion'].mean().sort_values(ascending=False)\n",
    "ax = sns.barplot(x=cat_avg.index, y=cat_avg.values, palette='viridis')\n",
    "plt.title('Calificación Promedio por Categoría')\n",
    "plt.xlabel('Categoría')\n",
    "plt.ylabel('Calificación Promedio')\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "# Añadir etiquetas de valor en las barras\n",
    "for i, v in enumerate(cat_avg.values):\n",
    "    ax.text(i, v + 0.1, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Gráfico de radar para visualizar las categorías\n",
    "def radar_chart(df):\n",
    "    # Calcular promedios por categoría\n",
    "    cat_avg = df.groupby('categoria')['calificacion'].mean()\n",
    "    \n",
    "    # Preparar datos para el gráfico de radar\n",
    "    categories = cat_avg.index.tolist()\n",
    "    values = cat_avg.values.tolist()\n",
    "    \n",
    "    # Cerrar el polígono repitiendo el primer valor\n",
    "    values.append(values[0])\n",
    "    categories.append(categories[0])\n",
    "    \n",
    "    # Calcular ángulos para cada categoría\n",
    "    N = len(categories) - 1  # -1 porque repetimos el primero\n",
    "    angles = [n / float(N) * 2 * np.pi for n in range(N)]\n",
    "    angles += [angles[0]]  # Cerrar el círculo\n",
    "    \n",
    "    # Crear figura\n",
    "    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))\n",
    "    \n",
    "    # Dibujar polígono\n",
    "    ax.plot(angles, values, linewidth=2, linestyle='solid', label='Promedio')\n",
    "    ax.fill(angles, values, alpha=0.25)\n",
    "    \n",
    "    # Añadir etiquetas\n",
    "    ax.set_thetagrids(np.degrees(angles), categories)\n",
    "    \n",
    "    # Configurar límites del eje radial\n",
    "    ax.set_ylim(0, 10)\n",
    "    ax.set_yticks(range(1, 11))\n",
    "    \n",
    "    # Añadir título\n",
    "    plt.title('Perfil Promedio de la Ruleta de la Vida', size=15, y=1.1)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "# Crear gráfico de radar\n",
    "radar_chart(df)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Análisis detallado por pregunta dentro de cada categoría\n",
    "for categoria in df['categoria'].unique():\n",
    "    # Filtrar datos para la categoría actual\n",
    "    cat_data = df[df['categoria'] == categoria]\n",
    "    \n",
    "    # Calcular estadísticas por pregunta\n",
    "    pregunta_stats = cat_data.groupby('pregunta')['calificacion'].agg(['mean', 'std']).reset_index()\n",
    "    \n",
    "    # Ordenar por calificación promedio\n",
    "    pregunta_stats = pregunta_stats.sort_values('mean', ascending=False)\n",
    "    \n",
    "    # Crear gráfico\n",
    "    plt.figure(figsize=(12, 6))\n",
    "    ax = sns.barplot(x='mean', y='pregunta', data=pregunta_stats, palette='Blues_d', \n",
    "                    xerr=pregunta_stats['std'], error_kw={'capsize': 5})\n",
    "    \n",
    "    # Añadir etiquetas de valor\n",
    "    for i, v in enumerate(pregunta_stats['mean']):\n",
    "        ax.text(v + 0.1, i, f'{v:.2f}', va='center')\n",
    "    \n",
    "    plt.title(f'Calificaciones Promedio por Pregunta - {categoria}')\n",
    "    plt.xlabel('Calificación Promedio')\n",
    "    plt.ylabel('Pregunta')\n",
    "    plt.xlim(0, 10.5)  # Establecer límites del eje x\n",
    "    plt.grid(True, alpha=0.3)\n",
    "    plt.tight_layout()\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Análisis Demográfico\n",
    "\n",
    "Analizamos cómo las calificaciones varían según factores demográficos como edad, sexo y estado civil."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calificaciones por grupo de edad\n",
    "plt.figure(figsize=(12, 6))\n",
    "edad_avg = df.groupby('grupo_edad')['calificacion'].mean().reset_index()\n",
    "ax = sns.barplot(x='grupo_edad', y='calificacion', data=edad_avg, palette='YlOrRd')\n",
    "plt.title('Calificación Promedio por Grupo de Edad')\n",
    "plt.xlabel('Grupo de Edad')\n",
    "plt.ylabel('Calificación Promedio')\n",
    "plt.ylim(0, 10)\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "# Añadir etiquetas de valor\n",
    "for i, v in enumerate(edad_avg['calificacion']):\n",
    "    ax.text(i, v + 0.1, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')\n",
    "\n",
    "plt.show()\n",
    "\n",
    "# Calificaciones por sexo\n",
    "plt.figure(figsize=(10, 6))\n",
    "sexo_avg = df.groupby('sexo')['calificacion'].mean().reset_index()\n",
    "ax = sns.barplot(x='sexo', y='calificacion', data=sexo_avg, palette='Set2')\n",
    "plt.title('Calificación Promedio por Sexo')\n",
    "plt.xlabel('Sexo')\n",
    "plt.ylabel('Calificación Promedio')\n",
    "plt.ylim(0, 10)\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "# Añadir etiquetas de valor\n",
    "for i, v in enumerate(sexo_avg['calificacion']):\n",
    "    ax.text(i, v + 0.1, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')\n",
    "\n",
    "plt.show()\n",
    "\n",
    "# Calificaciones por estado civil\n",
    "plt.figure(figsize=(12, 6))\n",
    "estado_avg = df.groupby('estado_civil')['calificacion'].mean().reset_index()\n",
    "ax = sns.barplot(x='estado_civil', y='calificacion', data=estado_avg, palette='Set3')\n",
    "plt.title('Calificación Promedio por Estado Civil')\n",
    "plt.xlabel('Estado Civil')\n",
    "plt.ylabel('Calificación Promedio')\n",
    "plt.ylim(0, 10)\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "# Añadir etiquetas de valor\n",
    "for i, v in enumerate(estado_avg['calificacion']):\n",
    "    ax.text(i, v + 0.1, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')\n",
    "\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Análisis cruzado: Categoría por Grupo de Edad\n",
    "plt.figure(figsize=(14, 8))\n",
    "cat_edad = df.groupby(['categoria', 'grupo_edad'])['calificacion'].mean().unstack()\n",
    "ax = cat_edad.plot(kind='bar', figsize=(14, 8), width=0.8)\n",
    "plt.title('Calificación Promedio por Categoría y Grupo de Edad')\n",
    "plt.xlabel('Categoría')\n",
    "plt.ylabel('Calificación Promedio')\n",
    "plt.legend(title='Grupo de Edad')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Análisis cruzado: Categoría por Sexo\n",
    "plt.figure(figsize=(14, 8))\n",
    "cat_sexo = df.groupby(['categoria', 'sexo'])['calificacion'].mean().unstack()\n",
    "ax = cat_sexo.plot(kind='bar', figsize=(14, 8), width=0.8)\n",
    "plt.title('Calificación Promedio por Categoría y Sexo')\n",
    "plt.xlabel('Categoría')\n",
    "plt.ylabel('Calificación Promedio')\n",
    "plt.legend(title='Sexo')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Análisis cruzado: Categoría por Estado Civil\n",
    "plt.figure(figsize=(14, 8))\n",
    "cat_estado = df.groupby(['categoria', 'estado_civil'])['calificacion'].mean().unstack()\n",
    "ax = cat_estado.plot(kind='bar', figsize=(14, 8), width=0.8)\n",
    "plt.title('Calificación Promedio por Categoría y Estado Civil')\n",
    "plt.xlabel('Categoría')\n",
    "plt.ylabel('Calificación Promedio')\n",
    "plt.legend(title='Estado Civil')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Heatmap de calificaciones por grupo de edad y categoría\n",
    "plt.figure(figsize=(14, 10))\n",
    "pivot_edad_cat = df.pivot_table(values='calificacion', index='grupo_edad', columns='categoria', aggfunc='mean')\n",
    "sns.heatmap(pivot_edad_cat, annot=True, cmap='YlGnBu', fmt='.2f', linewidths=0.5)\n",
    "plt.title('Calificaciones Promedio por Grupo de Edad y Categoría')\n",
    "plt.ylabel('Grupo de Edad')\n",
    "plt.xlabel('Categoría')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Heatmap de calificaciones por sexo y categoría\n",
    "plt.figure(figsize=(14, 8))\n",
    "pivot_sexo_cat = df.pivot_table(values='calificacion', index='sexo', columns='categoria', aggfunc='mean')\n",
    "sns.heatmap(pivot_sexo_cat, annot=True, cmap='YlGnBu', fmt='.2f', linewidths=0.5)\n",
    "plt.title('Calificaciones Promedio por Sexo y Categoría')\n",
    "plt.ylabel('Sexo')\n",
    "plt.xlabel('Categoría')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Análisis Temporal\n",
    "\n",
    "Analizamos cómo las calificaciones han evolucionado a lo largo del tiempo."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Tendencia temporal general\n",
    "tendencia_temporal = df.groupby('fecha_solo')['calificacion'].mean().reset_index()\n",
    "\n",
    "plt.figure(figsize=(14, 6))\n",
    "plt.plot(tendencia_temporal['fecha_solo'], tendencia_temporal['calificacion'], marker='o', linestyle='-', color='steelblue')\n",
    "plt.title('Tendencia de Calificaciones a lo Largo del Tiempo')\n",
    "plt.xlabel('Fecha')\n",
    "plt.ylabel('Calificación Promedio')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.ylim(0, 10)\n",
    "\n",
    "# Añadir línea de tendencia\n",
    "z = np.polyfit(range(len(tendencia_temporal)), tendencia_temporal['calificacion'], 1)\n",
    "p = np.poly1d(z)\n",
    "plt.plot(tendencia_temporal['fecha_solo'], p(range(len(tendencia_temporal))), \"r--\", \n",
    "         label=f'Tendencia: {z[0]:.4f}x + {z[1]:.4f}')\n",
    "plt.legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Tendencia temporal por categoría\n",
    "plt.figure(figsize=(14, 8))\n",
    "\n",
    "# Agrupar por mes para tener una visualización más clara\n",
    "df['año_mes'] = df['fecha'].dt.to_period('M')\n",
    "tendencia_cat = df.groupby(['año_mes', 'categoria'])['calificacion'].mean().unstack()\n",
    "\n",
    "# Convertir el índice de período a datetime para graficar\n",
    "tendencia_cat.index = tendencia_cat.index.to_timestamp()\n",
    "\n",
    "# Graficar tendencia por categoría\n",
    "ax = tendencia_cat.plot(figsize=(14, 8), marker='o')\n",
    "plt.title('Tendencia de Calificaciones por Categoría')\n",
    "plt.xlabel('Mes')\n",
    "plt.ylabel('Calificación Promedio')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.ylim(0, 10)\n",
    "plt.legend(title='Categoría', bbox_to_anchor=(1.05, 1), loc='upper left')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Análisis por día de la semana\n",
    "dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']\n",
    "df['nombre_dia'] = df['dia_semana'].apply(lambda x: dias[x])\n",
    "\n",
    "plt.figure(figsize=(12, 6))\n",
    "dia_avg = df.groupby('nombre_dia')['calificacion'].mean().reindex(dias)\n",
    "ax = sns.barplot(x=dia_avg.index, y=dia_avg.values, palette='Blues_d')\n",
    "plt.title('Calificación Promedio por Día de la Semana')\n",
    "plt.xlabel('Día de la Semana')\n",
    "plt.ylabel('Calificación Promedio')\n",
    "plt.ylim(0, 10)\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "# Añadir etiquetas de valor\n",
    "for i, v in enumerate(dia_avg.values):\n",
    "    ax.text(i, v + 0.1, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')\n",
    "\n",
    "plt.show()\n",
    "\n",
    "# Análisis por mes\n",
    "meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']\n",
    "df['nombre_mes'] = df['mes'].apply(lambda x: meses[x-1])\n",
    "\n",
    "plt.figure(figsize=(14, 6))\n",
    "mes_avg = df.groupby('nombre_mes')['calificacion'].mean()\n",
    "# Reordenar los meses cronológicamente\n",
    "mes_avg = mes_avg.reindex(meses)\n",
    "ax = sns.barplot(x=mes_avg.index, y=mes_avg.values, palette='YlGnBu')\n",
    "plt.title('Calificación Promedio por Mes')\n",
    "plt.xlabel('Mes')\n",
    "plt.ylabel('Calificación Promedio')\n",
    "plt.ylim(0, 10)\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "\n",
    "# Añadir etiquetas de valor\n",
    "for i, v in enumerate(mes_avg.values):\n",
    "    ax.text(i, v + 0.1, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Correlaciones y Relaciones\n",
    "\n",
    "Analizamos las correlaciones entre diferentes variables y cómo se relacionan entre sí."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Relación entre edad y calificación\n",
    "plt.figure(figsize=(12, 6))\n",
    "sns.scatterplot(data=df, x='edad', y='calificacion', alpha=0.5, hue='sexo', palette='Set1')\n",
    "plt.title('Relación entre Edad y Calificación')\n",
    "plt.xlabel('Edad')\n",
    "plt.ylabel('Calificación')\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "# Añadir línea de tendencia\n",
    "sns.regplot(data=df, x='edad', y='calificacion', scatter=False, color='black')\n",
    "\n",
    "plt.legend(title='Sexo')\n",
    "plt.show()\n",
    "\n",
    "# Correlación entre variables numéricas\n",
    "numeric_cols = ['calificacion', 'edad', 'mes', 'dia_semana', 'es_fin_semana']\n",
    "corr_matrix = df[numeric_cols].corr()\n",
    "\n",
    "plt.figure(figsize=(10, 8))\n",
    "sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)\n",
    "plt.title('Matriz de Correlación de Variables Numéricas')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Análisis de correlación entre categorías\n",
    "# Crear un DataFrame con promedios por usuario y categoría\n",
    "user_cat_avg = df.groupby(['usuario_id', 'categoria'])['calificacion'].mean().unstack()\n",
    "\n",
    "# Calcular matriz de correlación entre categorías\n",
    "cat_corr = user_cat_avg.corr()\n",
    "\n",
    "plt.figure(figsize=(12, 10))\n",
    "sns.heatmap(cat_corr, annot=True, cmap='RdBu_r', fmt='.2f', linewidths=0.5, vmin=-1, vmax=1)\n",
    "plt.title('Correlación entre Categorías')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Pairplot para visualizar relaciones entre categorías\n",
    "plt.figure(figsize=(16, 14))\n",
    "sns.pairplot(user_cat_avg, diag_kind='kde', plot_kws={'alpha': 0.6})\n",
    "plt.suptitle('Relaciones entre Categorías', y=1.02, fontsize=16)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Insights y Conclusiones\n",
    "\n",
    "Resumimos los principales hallazgos y conclusiones del análisis exploratorio."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Resumen de estadísticas clave\n",
    "print(\"=== RESUMEN DE ESTADÍSTICAS CLAVE ===\")\n",
    "print(f\"Total de respuestas analizadas: {len(df)}\")\n",
    "print(f\"Total de usuarios únicos: {df['usuario_id'].nunique()}\")\n",
    "print(f\"Rango de fechas: {df['fecha'].min().date()} a {df['fecha'].max().date()}\")\n",
    "print(f\"\\nCalificación promedio general: {df['calificacion'].mean():.2f}/10\")\n",
    "print(f\"Desviación estándar: {df['calificacion'].std():.2f}\")\n",
    "\n",
    "# Top 3 categorías con mayor calificación\n",
    "top_cats = df.groupby('categoria')['calificacion'].mean().sort_values(ascending=False).head(3)\n",
    "print(\"\\nTop 3 categorías con mayor calificación:\")\n",
    "for cat, val in top_cats.items():\n",
    "    print(f\"- {cat}: {val:.2f}/10\")\n",
    "\n",
    "# Top 3 categorías con menor calificación\n",
    "bottom_cats = df.groupby('categoria')['calificacion'].mean().sort_values().head(3)\n",
    "print(\"\\nTop 3 categorías con menor calificación:\")\n",
    "for cat, val in bottom_cats.items():\n",
    "    print(f\"- {cat}: {val:.2f}/10\")\n",
    "\n",
    "# Diferencias demográficas significativas\n",
    "print(\"\\nDiferencias demográficas significativas:\")\n",
    "\n",
    "# Por sexo\n",
    "sexo_diff = df.groupby('sexo')['calificacion'].mean().ptp()\n",
    "if sexo_diff > 0.5:  # Umbral arbitrario para considerar una diferencia significativa\n",
    "    print(f\"- Diferencia por sexo: {sexo_diff:.2f} puntos\")\n",
    "    for sexo, val in df.groupby('sexo')['calificacion'].mean().items():\n",
    "        print(f\"  * {sexo}: {val:.2f}/10\")\n",
    "\n",
    "# Por estado civil\n",
    "estado_diff = df.groupby('estado_civil')['calificacion'].mean().ptp()\n",
    "if estado_diff > 0.5:\n",
    "    print(f\"- Diferencia por estado civil: {estado_diff:.2f} puntos\")\n",
    "    for estado, val in df.groupby('estado_civil')['calificacion'].mean().sort_values(ascending=False).items():\n",
    "        print(f\"  * {estado}: {val:.2f}/10\")\n",
    "\n",
    "# Por grupo de edad\n",
    "edad_diff = df.groupby('grupo_edad')['calificacion'].mean().ptp()\n",
    "if edad_diff > 0.5:\n",
    "    print(f\"- Diferencia por grupo de edad: {edad_diff:.2f} puntos\")\n",
    "    for edad, val in df.groupby('grupo_edad')['calificacion'].mean().sort_values(ascending=False).items():\n",
    "        print(f\"  * {edad}: {val:.2f}/10\")\n",
    "\n",
    "# Tendencias temporales\n",
    "print(\"\\nTendencias temporales:\")\n",
    "# Calcular tendencia lineal\n",
    "tendencia_temporal = df.groupby('fecha_solo')['calificacion'].mean().reset_index()\n",
    "x = range(len(tendencia_temporal))\n",
    "y = tendencia_temporal['calificacion']\n",
    "z = np.polyfit(x, y, 1)\n",
    "pendiente = z[0]\n",
    "\n",
    "if abs(pendiente) > 0.01:  # Umbral arbitrario para considerar una tendencia significativa\n",
    "    tendencia = \"al alza\" if pendiente > 0 else \"a la baja\"\n",
    "    print(f\"- Tendencia general {tendencia} ({pendiente:.4f} puntos por día)\")\n",
    "else:\n",
    "    print(\"- No se observa una tendencia temporal significativa\")\n",
    "\n",
    "# Correlaciones importantes\n",
    "print(\"\\nCorrelaciones importantes entre categorías:\")\n",
    "# Filtrar correlaciones fuertes (positivas o negativas)\n",
    "strong_corr = []\n",
    "for i in range(len(cat_corr.columns)):\n",
    "    for j in range(i+1, len(cat_corr.columns)):\n",
    "        corr_val = cat_corr.iloc[i, j]\n",
    "        if abs(corr_val) > 0.6:  # Umbral arbitrario para correlación fuerte\n",
    "            strong_corr.append((cat_corr.columns[i], cat_corr.columns[j], corr_val))\n",
    "\n",
    "if strong_corr:\n",
    "    for cat1, cat2, corr in sorted(strong_corr, key=lambda x: abs(x[2]), reverse=True):\n",
    "        tipo = \"positiva\" if corr > 0 else \"negativa\"\n",
    "        print(f\"- Correlación {tipo} entre {cat1} y {cat2}: {corr:.2f}\")\n",
    "else:\n",
    "    print(\"- No se encontraron correlaciones fuertes entre categorías\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Conclusiones Generales\n",
    "\n",
    "Basándonos en el análisis exploratorio de datos, podemos extraer las siguientes conclusiones:\n",
    "\n",
    "1. **Distribución de Calificaciones**: Las calificaciones muestran una distribución centrada alrededor de [valor medio], con variaciones significativas entre categorías.\n",
    "\n",
    "2. **Categorías Destacadas**: \n",
    "   - Las categorías mejor evaluadas son [top categorías], lo que sugiere que los usuarios se sienten más satisfechos en estas áreas de sus vidas.\n",
    "   - Las categorías con menor puntuación son [categorías bajas], indicando áreas de oportunidad para mejora.\n",
    "\n",
    "3. **Factores Demográficos**:\n",
    "   - La edad muestra una [relación con edad] con las calificaciones, siendo los grupos de [grupos destacados] los que reportan mayor satisfacción.\n",
    "   - Existen diferencias notables entre sexos en categorías como [categorías con diferencias].\n",
    "   - El estado civil influye significativamente en categorías como [categorías afectadas], con [estado civil destacado] mostrando mayores puntuaciones.\n",
    "\n",
    "4. **Patrones Temporales**:\n",
    "   - Se observa una tendencia [tendencia] en las calificaciones a lo largo del tiempo.\n",
    "   - Existe estacionalidad con [meses/días] mostrando calificaciones consistentemente más altas.\n",
    "\n",
    "5. **Correlaciones**:\n",
    "   - Fuerte correlación positiva entre [categorías correlacionadas positivamente], sugiriendo que estas áreas tienden a mejorar juntas.\n",
    "   - Correlación negativa entre [categorías correlacionadas negativamente], indicando posibles compensaciones o trade-offs.\n",
    "\n",
    "6. **Recomendaciones para Análisis Futuro**:\n",
    "   - Profundizar en el análisis de [áreas de interés] mediante modelos predictivos.\n",
    "   - Realizar segmentación de usuarios para identificar perfiles distintos.\n",
    "   - Investigar factores externos que puedan influir en las tendencias temporales observadas.\n",
    "   - Diseñar intervenciones específicas para mejorar las categorías con puntuaciones más bajas.\n",
    "\n",
    "Este análisis exploratorio proporciona una base sólida para entender los patrones en los datos de la \"Ruleta de la Vida\" y puede informar tanto el desarrollo de modelos predictivos como la toma de decisiones para mejorar la experiencia de los usuarios."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

